# Detectron2 Beginner's Tutorial

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

Welcome to detectron2! This is the official colab tutorial of detectron2. Here, we will go through some basics usage of detectron2, including the following:
* Run inference on images or videos, with an existing detectron2 model
* Train a detectron2 model on a new dataset

You can make a copy of this tutorial by "File -> Open in playground mode" and make changes there. __DO NOT__ request access to this tutorial.


# Install detectron2

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6


In [ ]:
# Some basic setup:
from google.colab import drive
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Run a pre-trained detectron2 model

**Test on my own set of images**

**load images from your local file system**

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to the directory containing your images in Google Drive
drive_image_dir ="/content/drive/MyDrive/imgs_test_set"

In [ ]:
output_directory = "/content/drive/MyDrive/imgs_test_set/Detectron2_result"
# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [ ]:
# Create the Detectron2 configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model

# Load the pre-trained model weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# Create a predictor
predictor = DefaultPredictor(cfg)

[10/05 10:30:36 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


model_final_f10217.pkl: 178MB [00:00, 194MB/s]                           


**Save results**

In [ ]:
# List all image files in the directory in Google Drive
image_files = [os.path.join(drive_image_dir, filename) for filename in os.listdir(drive_image_dir) if filename.endswith(".jpg")]

for image_path in image_files:
    # Load an image from Google Drive
    im = cv2.imread(image_path)

    # Display the input image
    cv2_imshow(im)

    # Perform inference on the loaded image
    outputs = predictor(im)

    # Print predicted classes and boxes
    print(outputs["instances"].pred_classes)
    print(outputs["instances"].pred_boxes)

    # Visualize the predictions
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    # Get the filename of the input image
    input_image_filename = os.path.basename(image_path)

    # Define the path to save the result image
    result_image_path = os.path.join(output_directory, input_image_filename)

    # Save the image with predictions to the specified directory
    cv2.imwrite(result_image_path, out.get_image()[:, :, ::-1])

    # Display the image with predictions
    cv2_imshow(out.get_image()[:, :, ::-1])

    # Wait for a key press or a certain duration (e.g., 1 second)
    key = cv2.waitKey(1000)  # Adjust the duration as needed

    # Close the current image window
    cv2.destroyAllWindows()